<a href="https://colab.research.google.com/github/zideric/colab/blob/main/CNN_LSTM_architettureMiste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN + LSTM con Keras
proviamo a creare una rete neurale che combina uno stato convoluzionale con uno ricorrente per classificare le rencesioni di IMDB

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

## Caricamento e preprocessing del dataset
carichiamo il dataset con Keras con le 10k parole piu comuni, poi tronchiamo le recenzioni a 500 con pad_sequences

In [11]:
num_words = 10000
maxlen = 500

(X_train, y_train),(X_test,y_test) = imdb.load_data(num_words=num_words)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

## Creazione del modello

### Modello1: Rete convoluzionale

In [12]:
from keras.layers import Embedding, LSTM, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

model=Sequential()

model.add(Embedding(num_words, 50, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 500, 32)           4832      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 8001      
Total params: 512,833
Trainable params: 512,833
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512,validation_split=0.2, epochs=5)
model.evaluate(X_test,y_test)

Epoch 1/10
40/40 [==============================] - 17s 405ms/step - loss: 0.6851 - accuracy: 0.5438 - val_loss: 0.5720 - val_accuracy: 0.7052
Epoch 2/10
40/40 [==============================] - 16s 397ms/step - loss: 0.4749 - accuracy: 0.8108 - val_loss: 0.3626 - val_accuracy: 0.8440
Epoch 3/10
40/40 [==============================] - 16s 397ms/step - loss: 0.2905 - accuracy: 0.8869 - val_loss: 0.3556 - val_accuracy: 0.8504
Epoch 4/10
40/40 [==============================] - 16s 397ms/step - loss: 0.2251 - accuracy: 0.9152 - val_loss: 0.3291 - val_accuracy: 0.8614
Epoch 5/10
40/40 [==============================] - 16s 390ms/step - loss: 0.1845 - accuracy: 0.9294 - val_loss: 0.2643 - val_accuracy: 0.8918
Epoch 6/10
40/40 [==============================] - 16s 391ms/step - loss: 0.1580 - accuracy: 0.9415 - val_loss: 0.3342 - val_accuracy: 0.8740
Epoch 7/10
40/40 [==============================] - 16s 391ms/step - loss: 0.1339 - accuracy: 0.9530 - val_loss: 0.2884 - val_accuracy: 0.8936

[0.37882164120674133, 0.868120014667511]

### Modello2: da ricorrente a convoluzionale

In [14]:
model=Sequential()

model.add(Embedding(num_words, 50, input_length=maxlen))
model.add(LSTM(32, dropout=0.4, return_sequences=True))
model.add(Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 32)           10624     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 8001      
Total params: 521,729
Trainable params: 521,729
Non-trainable params: 0
________________________________________________

compiliamo e addestriamo per 10 epoche

In [15]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512,validation_split=0.2, epochs=5)
model.evaluate(X_test,y_test)

Epoch 1/10
40/40 [==============================] - 66s 2s/step - loss: 0.6613 - accuracy: 0.5772 - val_loss: 0.4519 - val_accuracy: 0.7998
Epoch 2/10
40/40 [==============================] - 65s 2s/step - loss: 0.3698 - accuracy: 0.8479 - val_loss: 0.3066 - val_accuracy: 0.8742
Epoch 3/10
40/40 [==============================] - 66s 2s/step - loss: 0.2854 - accuracy: 0.8854 - val_loss: 0.4981 - val_accuracy: 0.8296
Epoch 4/10
40/40 [==============================] - 64s 2s/step - loss: 0.3020 - accuracy: 0.8841 - val_loss: 0.2752 - val_accuracy: 0.8900
Epoch 5/10
40/40 [==============================] - 64s 2s/step - loss: 0.2040 - accuracy: 0.9227 - val_loss: 0.2778 - val_accuracy: 0.8912
Epoch 6/10
40/40 [==============================] - 65s 2s/step - loss: 0.1843 - accuracy: 0.9291 - val_loss: 0.3007 - val_accuracy: 0.8930
Epoch 7/10
40/40 [==============================] - 64s 2s/step - loss: 0.1781 - accuracy: 0.9333 - val_loss: 0.4032 - val_accuracy: 0.8556
Epoch 8/10
40/40 [==

[0.34626418352127075, 0.8763599991798401]

### Modello3: da convoluzionale a ricorrente
L'archietettura della rete sarà la seguente:
* primo strato embedding
* secondo strato convoluzionale con 32 filtri di dimensione 3*3
* terzo strato riduce la dimensione delle features map con il max pooling
* quarto strato è ricorrente LSTM, per ridurre overfitting eseguiamo dropout sull'input del 40% dei nodi
* quinto strato output della rete

usaimo il Conv1D perche in questo caso non siamo di fronte ad un tensore ma ad una semplice matrice (tensore sono piu matrici sovrapposte)

In [16]:
model=Sequential()

model.add(Embedding(num_words, 50, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(32, dropout=0.4))
# model.add(Flatten()) non serve perche output di lstm è già Flat se non settato return_sequences = True
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 500, 32)           4832      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 513,185
Trainable params: 513,185
Non-trainable params: 0
_________________________________________________________________


compiliamo ed eseguiamo l'addestramento


In [17]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512,validation_split=0.2, epochs=5)
model.evaluate(X_test,y_test)

Epoch 1/5
40/40 [==============================] - 40s 935ms/step - loss: 0.6676 - accuracy: 0.5984 - val_loss: 0.4801 - val_accuracy: 0.7880
Epoch 2/5
40/40 [==============================] - 37s 927ms/step - loss: 0.4062 - accuracy: 0.8366 - val_loss: 0.7757 - val_accuracy: 0.6474
Epoch 3/5
40/40 [==============================] - 36s 910ms/step - loss: 0.3283 - accuracy: 0.8721 - val_loss: 0.7532 - val_accuracy: 0.6854
Epoch 4/5
40/40 [==============================] - 36s 909ms/step - loss: 0.2785 - accuracy: 0.8973 - val_loss: 0.2808 - val_accuracy: 0.8840
Epoch 5/5
782/782 [==============================] - 24s 31ms/step - loss: 0.3109 - accuracy: 0.8738


[0.3109285533428192, 0.8737599849700928]

l'approccio ibrido, con diverse sperimentazioni, porta a risultati migliori